In [12]:
import scvelo as scv
from lets_plot import *
import polars as pl

LetsPlot.setup_html()

In [3]:
adata = scv.datasets.pancreas()

100%|██████████| 50.0M/50.0M [00:08<00:00, 5.94MB/s]


In [ ]:
scv.pp.filter_genes(adata, min_shared_counts=20)
scv.pp.normalize_per_cell(adata)
scv.pp.filter_genes_dispersion(adata, n_top_genes=2000)
scv.pp.log1p(adata)
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)

In [4]:
scv.tl.velocity(adata)

Normalized count data: X, spliced, unspliced.
computing neighbors


/Users/zaf4/Library/Caches/pypoetry/virtualenvs/cellestial-QESvkR8L-py3.12/lib/python3.12/site-packages/scvelo/tools/velocity.py:326: DeprecationWarning: Automatic neighbor calculation is deprecated since scvelo==0.4.0 and will be removed in a future version of scVelo. Please compute neighbors first with Scanpy.
  moments(adata)
/Users/zaf4/Library/Caches/pypoetry/virtualenvs/cellestial-QESvkR8L-py3.12/lib/python3.12/site-packages/scvelo/preprocessing/moments.py:71: DeprecationWarning: `neighbors` is deprecated since scvelo==0.4.0 and will be removed in a future version of scVelo. Please compute neighbors with Scanpy.
  neighbors(


    finished (0:00:03) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:01) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)
computing velocities


/Users/zaf4/Library/Caches/pypoetry/virtualenvs/cellestial-QESvkR8L-py3.12/lib/python3.12/site-packages/scvelo/tools/optimization.py:184: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  gamma[i] = np.linalg.pinv(A.T.dot(A)).dot(A.T.dot(y[:, i]))


    finished (0:00:04) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)


In [5]:
scv.tl.velocity_graph(adata)

computing velocity graph (using 1/10 cores)
or disable the progress bar using `show_progress_bar=False`.
    finished (0:00:15) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)


In [ ]:
scv.tools.velocity_embedding(adata, basis="umap")

computing velocity embedding
    finished (0:00:00) --> added
    'velocity_umap', embedded velocity vectors (adata.obsm)


In [ ]:
frame = (
    pl.from_numpy(adata.obsm["velocity_umap"], schema=["vel1", "vel2"])
    .with_columns(pl.from_numpy(adata.obsm["X_umap"], schema=["umap1", "umap2"]))
    .with_columns(pl.Series(adata.obs["clusters"]))
)

In [ ]:
frame.with_columns(
    pl.col("vel1").add(pl.col("umap1")).alias("vel__1"),
    pl.col("vel2").add(pl.col("umap2")).alias("vel__2"),
)

vel1,vel2,umap1,umap2,clusters,vel__1
f64,f64,f32,f32,cat,f64
-0.014429,0.048416,6.143066,-0.063644,"""Pre-endocrine""",6.128637
-0.030924,-0.034477,-9.906417,0.197778,"""Ductal""",-9.937341
-0.006753,0.007642,7.559791,0.583762,"""Alpha""",7.553038
-0.002182,0.038002,-11.283765,4.218998,"""Ductal""",-11.285946
0.131615,0.013901,1.721565,-4.753407,"""Ngn3 high EP""",1.85318
…,…,…,…,…,…
0.038975,0.071096,4.768472,-1.388353,"""Pre-endocrine""",4.807447
0.086775,-0.009781,-1.873335,-4.18265,"""Ngn3 high EP""",-1.78656
-0.035079,-0.020047,-9.88225,-0.105594,"""Ductal""",-9.917329


In [ ]:
base = (
    ggplot(frame)
    + geom_point(aes(x="umap1", y="umap2", color="clusters"), alpha=0.2)
    + scale_color_viridis()
    + ggsize(700, 400)
    + theme_void()
)
base

In [ ]:
base + geom_curve(
    aes("umap1", "umap2", xend="vel1", yend="vel2"),
    curvature=0.5,
    size=0.1,
    sampling=sampling_random(20),
)